# Answering Business Questions Using SQL

## Introduction 

This project will answer various business questions using SQL.

The database used is called Chinook, which is a sample database that represents a digital media store, including tables for artists, albums, media tracks, invoices, and customers.

* Media-related data was created using real data from an Apple iTunes library.
* Customer and employee information was created using fictitious names and addresses that can be located on Google maps, and other well formatted data (phone, fax, email, etc.)
* Sales information was auto generated using random data for a four year period.

The Chinook sample database includes:

* 11 tables
* A variety of indexes, primary and foreign key constraints
* Over 15,000 rows of data

A diagram of the databse schema can be found [here](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg).

Connecting the Jupyter Notebook to the database file:

In [4]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

Quering the database to get a list of all the tables and views in our database:

In [5]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table", "view")

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting New Albums to Purchase

Hypothetical Scenario:

The Chinook record store has just signed a deal with a new record label, and I've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

Artist Name and Genre
* Regal / Hip-Hop
* Red Tone / Punk
* Meteor and the Girls / Pop
* Slim Jim Bites / Blues

The record label specialises in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so I'm interested in finding out which genres sell the best in the USA.

In [66]:
%%sql

WITH USA_tracks_sold AS
    (SELECT il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
     WHERE country == "USA"
    )

SELECT
    g.name genre,
    COUNT(uts.quantity) absolute_tracks_sold,
    CAST(COUNT(uts.quantity) AS FLOAT) / (SELECT COUNT(*) FROM USA_tracks_sold) percentage_tracks_sold
FROM USA_tracks_sold uts
INNER JOIN track t ON t.track_id = uts.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10

Done.


genre,absolute_tracks_sold,percentage_tracks_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

* Red Tone (Punk)
* Meteor and Girls (Pop)
* Slim Jim Bites (Blues)

## Analysing Employee Sales Performance

Hypothetical Scenerio:

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. I have been asked to analyse the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [80]:
%%sql

WITH customer_support_sales AS
    (SELECT
         i.customer_id,
         c.support_rep_id,
         SUM(i.total) total
     FROM invoice i 
     INNER JOIN customer c ON i.customer_id = c.customer_id
     GROUP BY 1, 2
    )
    
SELECT 
    e.first_name || " " || e.last_name employee_name,
    e.hire_date,
    SUM(css.total) total_sales
FROM customer_support_sales css
INNER JOIN employee e ON e.employee_id = css.support_rep_id
GROUP BY 1

Done.


employee_name,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.5099999999998
Margaret Park,2017-05-03 00:00:00,1584.0000000000002
Steve Johnson,2017-10-17 00:00:00,1393.92


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

## Analysing Sales by Country 

Hypothetical Scenerio:

My next task is to analyse the sales data for customers from each different country. I have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, I have been directed to calculate data, for each country, on the:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

Because there are a number of countries with only one customer, you should group these customers as "Other" in your analysis. 

In [ ]:
%%sql

